**[中級機械学習 ホームページ](https://www.kaggle.com/learn/intermediate-machine-learning)**

---


ここからは**欠損値**の扱いについて学んだ知識を実際に試す番だ。

きっと大きな違いを実感できるはず。

# セットアップ

設問は作業内容にフィードバックを返す。
下のセルを実行してフィードバックシステムをセットアップしよう。

In [25]:
# # コードチェック用のセットアップ
# # 採点システムを有効化するためのコード
# # Set up code checking
# from learntools.core import binder
# binder.bind(globals())
# from learntools.ml_intermediate.ex2 import *
# print("Setup Complete")

この演習では、[Kaggle Learnユーザー向け住宅価格コンペティション](https://www.kaggle.com/c/home-data-for-ml-course)のデータを使う。

![Ames Housing データセット画像](https://i.imgur.com/lTJVG4e.png)

次のコードセルはそのまま実行しよう。
`X_train`, `X_valid`, `y_train`, `y_valid` に訓練・検証データが、`X_test` にテストデータが読み込まれる。

In [26]:
# --- データファイルの展開と確認 ---
# !ls ./input/02-missing-values
#   データディレクトリ内のファイル一覧を表示する。
!ls ./input/02-missing-values 
# !echo "---------------"
!echo "---------------"
# !gzip -d -c ./input/02-missing-values/train.csv.gz > ./input/02-missing-values/train.csv
#   train.csv.gz（圧縮ファイル）を解凍してtrain.csvを作成する。
!gzip -d -c ./input/02-missing-values/train.csv.gz > ./input/02-missing-values/train.csv
# !gzip -d -c ./input/02-missing-values/test.csv.gz > ./input/02-missing-values/test.csv
#   test.csv.gz（圧縮ファイル）を解凍してtest.csvを作成する。
!gzip -d -c ./input/02-missing-values/test.csv.gz > ./input/02-missing-values/test.csv
# !ls ./input/02-missing-values
#   再度ファイル一覧を表示し、解凍後のファイルが存在するか確認する。
!ls ./input/02-missing-values 


test.csv  test.csv.gz  train.csv  train.csv.gz
---------------


test.csv  test.csv.gz  train.csv  train.csv.gz


In [27]:
# --- pandas, scikit-learnのインポート ---
import pandas as pd  # データ操作・前処理のためのライブラリ
from sklearn.model_selection import train_test_split  # データ分割用

# --- データの読み込み ---
# train.csv（訓練データ）とtest.csv（テストデータ）をpandasで読み込む。
# index_col='Id' でId列をインデックスとして使う。
X_full = pd.read_csv('./input/02-missing-values/train.csv', index_col='Id')
X_test_full = pd.read_csv('./input/02-missing-values/test.csv', index_col='Id')

# --- 目的変数（SalePrice）と説明変数の分離 ---
# SalePrice（住宅価格）が欠損している行は分析対象外とする。
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
# 目的変数（ターゲット）yにSalePriceを格納。
y = X_full.SalePrice
# 説明変数（特徴量）からSalePrice列を除外。
X_full.drop(['SalePrice'], axis=1, inplace=True)

# --- 数値データのみ抽出 ---
# 機械学習の入門では、まずobject型（文字列など）を除外し、数値型のみで学習するのが簡単。
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# --- 訓練データと検証データへの分割 ---
# train_test_splitで、訓練用（X_train, y_train）と検証用（X_valid, y_valid）に分ける。
# train_size=0.8で8割を訓練、2割を検証に使う。
# random_state=0で分割の再現性を確保。
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

次のコードセルで、訓練データ（X_train）の先頭5行を表示しよう。

pandasの .head() メソッドは、データフレームの最初の5行を簡単に確認できる便利な関数。
データの中身やカラム名、値の型を把握するのに役立つ。

In [28]:
# --- データの先頭5行を表示 ---
# .head() でデータフレームの最初の5行を表示できる。
# データの中身やカラム名、値の型を把握するのに役立つ。
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


先ほど表示したデータの中にも、すでにいくつか欠損値（NaN）が含まれているのが分かる。

次のステップでは、データセット全体の欠損値の状況をより詳しく調べていく。

# ステップ1: 欠損値の事前調査

下のコードセルはそのまま実行しよう。

In [29]:
# --- データの形状と欠損値のカウント ---
# X_train.shape で（行数, 列数）を表示。
# 欠損値（NaN）がどのカラムにどれだけ含まれているかを調べる。
print(X_train.shape)

# 各カラムごとの欠損値の個数を計算。
missing_val_count_by_column = (X_train.isnull().sum())
# 欠損値が1つ以上あるカラムだけを表示。
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [30]:
# --- 欠損値の有無をTrue/Falseで判定 ---
isnull_df = X_train.isnull()
print(isnull_df.head())  # 各セルが欠損値ならTrue, そうでなければFalse

     MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
Id                                                                           
619       False        False    False        False        False      False   
871       False        False    False        False        False      False   
93        False        False    False        False        False      False   
818       False         True    False        False        False      False   
303       False        False    False        False        False      False   

     YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  ...  GarageArea  \
Id                                                     ...               
619         False       False       False       False  ...       False   
871         False       False       False       False  ...       False   
93          False       False       False       False  ...       False   
818         False       False       False       False  ...       False   
303      

上記のコードは、X_trainの各セルが欠損値（NaN）かどうかを判定し、True/Falseのデータフレームを返す。
pandasのisnull()は、各要素がNaNならTrue、そうでなければFalseになる。

In [31]:
# --- 各カラムごとの欠損値の個数をカウント ---
missing_val_count_by_column = isnull_df.sum()
print(missing_val_count_by_column)  # 各カラムごとの欠損値の個数

MSSubClass         0
LotFrontage      212
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         6
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       58
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
dtype: int64


isnull_df.sum() は、各カラムごとにTrue（=1）の数を合計する。
つまり、各カラムに何個の欠損値があるかをSeriesで返す。

In [32]:
# --- 欠損値が1つ以上あるカラムだけを抽出 ---
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


missing_val_count_by_column[missing_val_count_by_column > 0] で、
欠損値が1つ以上あるカラムだけを抽出して表示できる。

### パートA

上の出力結果を使って、以下の質問に答えよう。

- 訓練データの行数はいくつか？
- 欠損値を含むカラムはいくつか？
- 全体で欠損しているデータはいくつか？

（ヒント：print(X_train.shape)や、欠損値カウントの出力をよく見よう）

In [33]:
# # --- 訓練データの行数 ---
# # X_train.shape[0] で行数（サンプル数）が分かる。
# num_rows = 1168  # 行数

# # --- 欠損値を含むカラム数 ---
# # 欠損値が1つ以上あるカラムの数。
# num_cols_with_missing = 3  # 欠損カラム数

# # --- 全体の欠損値の個数 ---
# # 欠損値カウントの合計。
# tot_missing = 276  # 欠損値の総数

# # --- 答え合わせ（Kaggleの自動採点用）---
# step_1.a.check()

### パートB
上記の結果を踏まえて、欠損値への対処として最も良い方法は何だと思う？

+ **自分の答え:** 欠損しているデータが少ないので、削除よりも補完（imputation）が良い選択肢。

In [35]:
# step_1.b.hint()

In [36]:
# step_1.b.solution()

欠損値への対処法を比較するために、チュートリアルで使った `score_dataset()` 関数をここでも利用する。
この関数はランダムフォレストモデルでの[平均絶対誤差（MAE）](https://en.wikipedia.org/wiki/Mean_absolute_error)を計算する。

intermediate_machine_learning/02-missing-values.ipynb にそのまま貼り付けて使える、MAE（平均絶対誤差）を使う理由の詳細な解説（Markdownセル用）を出力した。

---

## なぜMAE（平均絶対誤差）を使うのか？

MAE（Mean Absolute Error）は、回帰問題でモデルの予測精度を評価するための指標の一つ。ここでMAEを使う理由を、他の指標（MSE, RMSEなど）と比較しながら詳しく解説する。

---

### 1. MAEの直感的な意味

- MAEは「予測値と実際の値のズレ（誤差）」の絶対値の平均。
- たとえばMAE=5なら「平均して5だけ外れている」と解釈できる。
- 単位も元データと同じなので、解釈しやすい。

---

### 2. 他の指標との比較

#### MSE（平均二乗誤差）・RMSE（二乗平均平方根誤差）

- MSEやRMSEは誤差を2乗するため、大きな外れ値（outlier）に非常に敏感。
- 外れ値が多いデータや、外れ値の影響を抑えたい場合には不向き。
- 単位が元データと異なり、直感的な解釈が難しい。

#### MAEの特徴

- 誤差をそのまま絶対値で評価するため、外れ値の影響を受けにくい。
- 「平均的にどれくらい外れているか」をそのまま示す。
- 欠損値処理の比較など、シンプルな評価をしたいときに適している。

---

### 3. 欠損値処理の比較におけるMAEの利点

- 欠損値の補完方法を比較する際、モデルの予測誤差が「どれくらいズレているか」を直接知りたい。
- MAEは外れ値の影響を受けにくく、補完方法の違いによる「全体的なズレ」を素直に評価できる。
- そのため、欠損値処理の効果を比較する場面ではMAEがよく使われる。

---

### 4. まとめ

- MAEは「平均的なズレ」を直感的に示す指標。
- 外れ値の影響を受けにくく、解釈しやすい。
- 欠損値処理の比較など、シンプルな評価をしたいときに最適。

---

#### 参考: MAEの数式

\[
\mathrm{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
\]

- \( y_i \)：実際の値
- \( \hat{y}_i \)：予測値
- \( n \)：データ数

In [37]:
# --- モデル評価用関数の定義 ---
from sklearn.ensemble import RandomForestRegressor  # ランダムフォレスト回帰モデル
from sklearn.metrics import mean_absolute_error     # 平均絶対誤差（MAE）

# 様々な前処理方法の性能を比較するための関数。
# ランダムフォレストで学習し、検証データでのMAE（平均絶対誤差）を返す。
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)  # 決定木100本、乱数固定
    model.fit(X_train, y_train)  # 学習
    preds = model.predict(X_valid)  # 予測
    return mean_absolute_error(y_valid, preds)  # MAEを返す

## `score_dataset` 関数の解説

```python
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)  # 決定木100本、乱数固定
    model.fit(X_train, y_train)  # 学習
    preds = model.predict(X_valid)  # 予測
    return mean_absolute_error(y_valid, preds)  # MAEを返す
```

---

### 各ステップの説明

1. **モデルの作成**
    ```python
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    ```
    - ランダムフォレスト回帰モデルを作成。
    - `n_estimators=100`：100本の決定木を使う。
    - `random_state=0`：乱数シードを固定し、毎回同じ結果になるようにする。

2. **モデルの学習**
    ```python
    model.fit(X_train, y_train)
    ```
    - 学習用データ（特徴量 `X_train` と正解ラベル `y_train`）でモデルを訓練。

3. **予測**
    ```python
    preds = model.predict(X_valid)
    ```
    - 検証用データ（`X_valid`）に対して予測値を出す。

4. **評価（MAEの計算）**
    ```python
    return mean_absolute_error(y_valid, preds)
    ```
    - 予測値 `preds` と実際の値 `y_valid` の間の平均絶対誤差（MAE）を計算し、その値を返す。
    - MAEは「平均してどれくらい外れているか」を示す指標。

---

### ランダムフォレストとは

- ランダムフォレストは「複数の決定木を組み合わせて予測する」アンサンブル学習手法。
- 各決定木はデータの一部や特徴量の一部だけを使って学習し、それぞれ独立に予測を行う。
- 最終的な予測は、各決定木の予測結果の平均（回帰の場合）や多数決（分類の場合）で決まる。
- これにより、単一の決定木よりも「過学習しにくく」「精度が高い」モデルになる。

---

### 決定木とは

- 決定木は「特徴量に基づいてデータを分割し、最終的に予測値を出す」シンプルなモデル。
- 木構造で分岐を繰り返し、各葉ノードで予測値を決定する。
- 直感的で解釈しやすいが、単体では過学習しやすいという弱点がある。

---

### なぜ100本の決定木なのか

- 決定木の本数（`n_estimators`）を増やすことで、ランダムフォレストの予測はより安定し、精度が向上する。
- ただし、木の本数が多すぎると計算コストが増える。
- 100本は「十分な精度」と「計算コスト」のバランスが良いとされる一般的な値。
- 実際のタスクやデータセットによって最適な本数は異なるが、まずは100本程度から試すのが定石。

---

### まとめ

- この関数は「与えられたデータセットでランダムフォレスト回帰モデルを訓練し、検証データでのMAE（平均絶対誤差）を返す」もの。
- 欠損値処理などの前後でこの関数を使うことで、処理方法ごとのモデル精度（誤差）の違いを簡単に比較できる。
- ランダムフォレストは複数の決定木を使うことで、単一の決定木よりも高い汎化性能を持つ。
- 100本の決定木は、精度と計算コストのバランスを考慮した一般的な選択肢。

## ステップ2: 欠損値を含むカラムの除去

このステップでは、訓練データ（`X_train`）と検証データ（`X_valid`）から、1つでも欠損値（NaN）が含まれるカラムを全て除去する。
欠損値を含む特徴量を丸ごと削除することで、モデルが欠損値に悩まされずに学習できるようにする。
ただし、情報が失われるため、他の方法（補完など）と比較して精度がどう変わるかも確認する必要がある。

In [40]:
# --- 欠損値を含むカラムの除去 ---
# X_trainの各カラムについて、isnull().any() で1つでも欠損値があるかを判定し、
# 欠損値を含むカラム名をリストアップする。
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

# 上記のカラムを訓練・検証データから除去
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

# 除去したカラム名と、除去後のカラム数を確認
print("欠損値を含むカラム:", cols_with_missing)
print("除去後のカラム数:", reduced_X_train.shape[1])

# --- 答え合わせ（Kaggleの自動採点用）---
# このセルはKaggle環境でのみ有効。ローカル環境ではコメントアウトのままにしておく。
# step_2.check()

欠損値を含むカラム: ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
除去後のカラム数: 33


#### 【解説】リスト内包表記の分解と出力例

`cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]` の各構成要素を分解して解説する。

**X_train.columns は何か？**

`X_train.columns` は、訓練データ `X_train` の全てのカラム名（列名）を表す。型は pandas の Index 型。

【出力例】

In [42]:
print(X_train.columns)

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')


**for col in X_train.columns の目的**

`for col in X_train.columns` は、全てのカラム名（列名）を1つずつ順番に `col` という変数に取り出すためのループ。

これを使うことで、全カラムに対して同じ処理を繰り返し実行できる。

【出力例】

In [43]:
for col in list(X_train.columns)[:5]:
    print(col)

MSSubClass
LotFrontage
LotArea
OverallQual
OverallCond


**X_train[col].isnull() とは？**

`X_train[col].isnull()` は、カラム `col` の各要素が欠損値（NaN）かどうかを判定し、True/False の Series を返す。

【出力例】

In [44]:
print(X_train["LotFrontage"].isnull().head())  # 例として 'LotFrontage' カラム

Id
619    False
871    False
93     False
818     True
303    False
Name: LotFrontage, dtype: bool


**X_train[col].isnull().any() とは？**

`X_train[col].isnull().any()` は、カラム `col` の中に1つでも欠損値（NaN）があれば True、なければ False を返す。

【出力例】

In [45]:
print(X_train["LotFrontage"].isnull().any())  # 例として 'LotFrontage' カラム

True


**[col for col in X_train.columns if X_train[col].isnull().any()] の目的と出力例**

このリスト内包表記は「X_trainの全カラムのうち、1つでも欠損値（NaN）が含まれるカラム名だけをリストアップする」ためのもの。

【出力例】

In [46]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
print(cols_with_missing)

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']


---

In [41]:
# --- ヒントや解答を表示したい場合は、下のコメントアウトを外す ---
# step_2.hint()    # ヒントを表示
# step_2.solution() # 解答例を表示

上記コードでは、まず `X_train` の各カラムについて `isnull().any()` で欠損値が1つでもあるかを判定し、該当するカラム名をリスト `cols_with_missing` に格納している。
その後、`drop` メソッドでこれらのカラムを訓練・検証データから除去している。
最後に、除去後のカラム数を `shape[1]` で確認している。

【出力例】
```
欠損値を含むカラム: ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
除去後のカラム数: 33
```


# ステップ3: 補完（Imputation）

## 欠損値補完の概要

欠損値補完とは、欠損値（NaN）を何らかの値で埋める処理のこと。この方法では、データを削除せずに全ての情報を活用できる。

補完方法にはいくつかあるが、ここでは「平均値補完」を試す。

### パートA

次のコードセルでは、各カラムの平均値で欠損値を補完（impute）する。
補完後のデータフレームは `imputed_X_train` と `imputed_X_valid` に格納する。
カラム名が元の `X_train`, `X_valid` と一致していることを確認しよう。

In [ ]:
# --- 平均値での補完（Imputation）に必要なライブラリをインポート ---
from sklearn.impute import SimpleImputer  # 欠損値補完用

# SimpleImputerを使って、各カラムの平均値で欠損値を埋める
# strategy='mean'がデフォルトで平均値補完を意味する
my_imputer = SimpleImputer()  # デフォルトで平均値補完

# 訓練データに対して補完器を適合(fit)させ、変換(transform)する
# fit_transform()は「学習と変換」を一度に行う
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))

# 検証データは訓練データと同じ方法（同じ平均値）で補完する
# 検証データに対しては学習済みの補完器でtransformのみ実行
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))


#### なぜfitとtransformを分けるのか
訓練データと検証データで別々に処理している理由：

- データリーク防止: 検証データの情報が訓練に漏れないようにするため
- 一貫性の確保: 両方のデータセットで同じ値（訓練データから計算した平均値）を使って補完するため

例えば、訓練データのLotFrontageの平均が69.614017、検証データの平均が75.0だとしても、検証データの欠損値も69.614017で補完する。これにより、モデルが訓練時に見た値の分布と検証時の値の分布が一致し、公平な評価ができる。

コード例では、fit_transform()で訓練データに対して学習と変換を一度に行い、検証データにはtransform()のみを適用して、訓練データから学習した同じ平均値で補完している。

## SimpleImputerの動作と注意点

SimpleImputerは欠損値を補完するが、DataFrame→Numpy配列→DataFrameという変換過程でカラム名が失われる。そのため、元のカラム名を再設定する必要がある。

In [50]:

# SimpleImputerで補完するとカラム名が消えるので、元のカラム名を再設定
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


# 補完後のデータを確認（先頭5行）
imputed_X_train.head()


# --- 答え合わせ（Kaggleの自動採点用）---
# step_3.a.check()


# --- ヒントや解答を表示したい場合は、下のコメントアウトを外す ---
# step_3.a.hint()    # ヒントを表示
# step_3.a.solution() # 解答例を表示

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,20.0,90.000000,11694.0,9.0,5.0,2007.0,2007.0,452.0,48.0,0.0,...,774.0,0.0,108.0,0.0,0.0,260.0,0.0,0.0,7.0,2007.0
1,20.0,60.000000,6600.0,5.0,5.0,1962.0,1962.0,0.0,0.0,0.0,...,308.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2009.0
2,30.0,80.000000,13360.0,5.0,7.0,1921.0,2006.0,0.0,713.0,0.0,...,432.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,8.0,2009.0
3,20.0,69.614017,13265.0,8.0,5.0,2002.0,2002.0,148.0,1218.0,0.0,...,857.0,150.0,59.0,0.0,0.0,0.0,0.0,0.0,7.0,2008.0
4,20.0,118.000000,13704.0,7.0,5.0,2001.0,2002.0,150.0,0.0,0.0,...,843.0,468.0,81.0,0.0,0.0,0.0,0.0,0.0,1.0,2006.0



## 平均値補完後のモデル評価

補完したデータを使ってモデルを評価し、MAE（平均絶対誤差）を確認する。

In [51]:
# --- 平均値補完後のモデル評価（MAE算出）---
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))  # MAEを表示

MAE (Imputation):
18062.894611872147


### パートB

各手法（カラム削除と平均値補完）で得られたMAEを比較し、どちらが良い結果になったかを考察する。

In [52]:
# --- ヒントや解答を表示したい場合は、下のコメントアウトを外す ---
# step_3.b.hint()    # ヒントを表示
# step_3.b.solution() # 解答例を表示

# ステップ4: テストデータで予測を生成

## 最終的な欠損値処理方法の選択

この最終ステップでは、中央値（median）補完を採用する。中央値は外れ値の影響を受けにくいため、平均値よりも堅牢な補完方法となる場合が多い。

### パートA

訓練・検証データの最終前処理を行う。

次のコードセルで訓練・検証データを前処理し、
`final_X_train` と `final_X_valid` に格納する。
（下記の条件を満たす必要がある）
- 前処理後のデータフレームのカラム数が一致していること
- 欠損値が残っていないこと
- `final_X_train`と`y_train`の行数が一致していること
- `final_X_valid`と`y_valid`の行数が一致していること

In [53]:
# --- 訓練・検証データの最終前処理 ---
# SimpleImputerで中央値（median）補完を採用
# strategy='median' で各カラムの中央値で欠損値を埋める
final_imputer = SimpleImputer(strategy='median')

# 訓練データに対して補完器を適合(fit)させ、変換(transform)する
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))

# 検証データは訓練データと同じ方法（同じ中央値）で補完する
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

# SimpleImputerで補完するとカラム名が消えるので、元のカラム名を再設定
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

# --- 答え合わせ（Kaggleの自動採点用）---
# step_4.a.check()


## ランダムフォレストモデルの学習と評価

前処理したデータを使ってランダムフォレストモデルを学習し、検証データで評価する。
（この後テストデータの予測に使うため、ここでは `score_dataset()` 関数は使わない点に注意）


In [54]:
# --- ランダムフォレストで学習・評価 ---
# n_estimators=210 で決定木の本数を増やし、より安定した予測を目指す
# 決定木の本数を増やすと、一般的に予測精度が向上する（ただし計算コストも増加）
model = RandomForestRegressor(n_estimators=210, random_state=0)

# 訓練データでモデルを学習
model.fit(final_X_train, y_train)

# 検証データで予測
preds_valid = model.predict(final_X_valid)

# MAE（平均絶対誤差）を算出
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17459.893781256797


### パートB

テストデータも同じ方法で前処理し、学習済みモデルで予測を行う。

次のコードセルでテストデータ（X_test）も前処理し、
訓練・検証データと同じ方法（中央値補完）で処理して、
前処理済みのテストデータを `final_X_test` に格納する。

その後、学習済みモデルでテストデータの予測値を生成する。

（正解判定の条件）
- 欠損値が残っていないこと
- `final_X_test` の行数が `X_test` と一致していること


In [55]:
# --- テストデータの前処理 ---
# 訓練・検証データと同じ方法（中央値補完）でテストデータも処理
# 既に学習済みのfinal_imputerを使用することで、訓練データと同じ中央値で補完される
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# カラム名を再設定
final_X_test.columns = X_test.columns

# 学習済みモデルでテストデータの予測値を生成
preds_test = model.predict(final_X_test)

# --- 答え合わせ（Kaggleの自動採点用）---
# step_4.b.check()

## 予測結果の保存

Kaggleコンペに提出できる形式でCSVファイルに保存する。

```python
# --- 予測結果をCSVファイルに保存 ---
# Kaggleコンペの提出形式に合わせて、Id列とSalePrice列を持つDataFrameを作成
output = pd.DataFrame({'Id': X_test.index,
                      'SalePrice': preds_test})

# CSVファイルとして保存（index=Falseでインデックスは保存しない）
output.to_csv('submission.csv', index=False)
```

## ハイパーパラメータチューニングの例（参考）

モデルの性能をさらに向上させるために、決定木の本数（n_estimators）を変えて実験することもできる。

In [56]:
# 決定木の本数を変えながらMAEを確認する例（コメントアウト状態）
#for n_estimators in range(180,220):
#    model = RandomForestRegressor(n_estimators=n_estimators, random_state=0)
#    model.fit(final_X_train, y_train)
#
#    # Get validation predictions and MAE
#    preds_valid = model.predict(final_X_valid)
#    print("MAE (Your approach):", n_estimators, end=" --> ")
#    print(mean_absolute_error(y_valid, preds_valid))


# ステップ5: 結果の提出

Kaggleのリーダーボードに結果を提出する手順を説明する。

1. コンペティションに参加する（まだ参加していない場合）
2. 「COMMIT」ボタンをクリックしてコードを実行
3. 「Open Version」ボタンをクリックして結果を確認
4. 「Output」タブから「Submit to Competition」ボタンをクリックして提出
5. 改善したい場合は「Edit」ボタンをクリックして再度編集

# 次のステップ

次の章では「カテゴリ変数」について学ぶ。カテゴリ変数は実データでよく登場するが、前処理せずにモデルへ入力するとエラーになるので注意が必要。